### Initialize Client
Using the GA4GH client library, to ensure that the request is being formulated in acordance with the GA4GH requirements

In [1]:
import ga4gh.client as client
c = client.HttpClient("http://127.0.0.1:8000/data/ga4gh")

### Obtain available datasets
Datasets is the highest data level, and the info returned can be use to recursively access the lowest data level. That is, each individual variant.

In [2]:
dataset = c.search_datasets().next()
print dataset

id: "brca"
name: "brca-exchange-variants"
description: "Variants observed in brca-exchange project"



### Data set by id method
Observe that we can obtain a given dataset, just by knowing its id. Observe that on the previous instance we obtained all the available sets. Currently only one is supported.

In [3]:
individual_dataset = c.get_dataset(dataset_id="brca")
print individual_dataset

id: "brca"
name: "brca-exchange-variants"
description: "Variants observed in brca-exchange project"



### Obtaining Variant Sets
Note that by querying datasets, we obtain the variants id so that we can 
make a query to the variant sets and become more specific with which set we want our variants to be build and obtained.

In [4]:
# `dataset.id` obtained in search datasets call # 
variant_sets = [i for i in c.search_variant_sets(dataset_id=dataset.id)]
Sets = {}
for variantSets in variant_sets:
    Sets[variantSets.id] = {"Name" : variantSets.name, "Reference Set Id" : variantSets.reference_set_id,
                            "Data Set Id" : variantSets.dataset_id}
    print"Variant Set Id: {}\n\tName: {}\n\tReference Set Id: {}\n\tData Set Id: {}\n".format(variantSets.id,
        variantSets.name, variantSets.reference_set_id, variantSets.dataset_id)

Variant Set Id: brca-hg36
	Name: brca-exchange-variants-hg36
	Reference Set Id: Genomic-Coordinate-hg36
	Data Set Id: brca

Variant Set Id: brca-hg37
	Name: brca-exchange-variants-hg37
	Reference Set Id: Genomic-Coordinate-hg37
	Data Set Id: brca

Variant Set Id: brca-hg38
	Name: brca-exchange-variants-hg38
	Reference Set Id: Genomic-Coordinate-hg38
	Data Set Id: brca



##### Note, only selected fields where shown for illustration purposes. Because the independent variantset search will provide other informational parameters, stored as metadata, each with a individual description.

### Get variant set by id method
We can also call a specific set of variants by knowing its id. Currently the distinction of such is derrived from the supported genomic coordinates, so only 3 sets are available. GA4GH supports coordinate hg37, so we will use such to demonstrate the get variantset function 

In [5]:
Varset = c.get_variant_set(variant_set_id="brca-hg37")
print "Variant Id: {}\nName: {}\nDataset Id: {}\nReference Set Id: {}\n".format(Varset.id, Varset.name, Varset.dataset_id, Varset.reference_set_id)
for i in Varset.metadata:
    print "Metadata Field: {} ;  Value: {} ;  Type: {}".format(i.key, i.value, i.type)

Variant Id: brca-hg37
Name: brca-exchange-variants-hg37
Dataset Id: brca
Reference Set Id: Genomic-Coordinate-hg37

Metadata Field: id ;  Value: - ;  Type: AutoField
Metadata Field: Variant_in_ENIGMA ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_ClinVar ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_1000_Genomes ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_ExAC ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_LOVD ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_BIC ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_ESP ;  Value: - ;  Type: BooleanField
Metadata Field: Variant_in_exLOVD ;  Value: - ;  Type: BooleanField
Metadata Field: Source ;  Value: - ;  Type: TextField
Metadata Field: URL_ENIGMA ;  Value: - ;  Type: TextField
Metadata Field: Condition_ID_type_ENIGMA ;  Value: - ;  Type: TextField
Metadata Field: Condition_ID_value_ENIGMA ;  Value: - ;  Type: TextField
Metadata Field: Condition_category_E

### Searching variants
Knowing some variant set id obtained in the previous call and the reference name, we can make a taylorized search for variants and obtain the ones which comply with our request.


In [6]:
counter = 0
top = 55
variants_in_range = [i for i in c.search_variants(reference_name="chr17", variant_set_id="brca-hg37", start=41246794, end=41296814)]
print "Variants in selected range: {}\n".format(len(variants_in_range))
for Vars in variants_in_range:
    if 45 < counter < top:
        if Vars.info["Clinical_Significance_ClinVar"].values:
            SigVar = Vars.info["Pathogenicity_default"].values[0].string_value
        else : SigVar = "None"
        print "Variant Id: {},\tVariant Set Id: {},\tReference Name: {}\n\tVariant Start: {},\tVariant End: {}\n\tReference Bases: {},\tAlternate Bases: {},\tClinical Significance: {}\n".format(Vars.id, Vars.variant_set_id,Vars.reference_name,Vars.start, Vars.end,Vars.reference_bases,Vars.alternate_bases,SigVar)
    if counter >= top:
        break
    counter += 1

Variants in selected range: 1533

Variant Id: hg37-12112,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41246924,	Variant End: 41246926
	Reference Bases: T,	Alternate Bases: [u'TA'],	Clinical Significance: None

Variant Id: hg37-12682,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41246925,	Variant End: 41246926
	Reference Bases: A,	Alternate Bases: [u'T'],	Clinical Significance: None

Variant Id: hg37-3325,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41246974,	Variant End: 41246975
	Reference Bases: C,	Alternate Bases: [u'G'],	Clinical Significance: None

Variant Id: hg37-5680,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41246981,	Variant End: 41246982
	Reference Bases: A,	Alternate Bases: [u'G'],	Clinical Significance: None

Variant Id: hg37-10814,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41247004,	Variant End: 41247005
	Reference Bases: C,	Alternate Bases: [u'G'],	Clinical Signifi

##### Note, obsereve that only a selected amount of parameters where chosen to be desplayed under variants search method. But the metadata fields which are defined are also available in this request. Potential fields available are defined under "Get variant set by id method" example 

### Get variant by id
Observe that in the above example we obtain id's for the variants available, which are contained within the requested genomic range. Also, other fields which are present are also displayed in the call.  

In [7]:
SingleVar = c.get_variant(variant_id="hg37-398")
SigVar = SingleVar.info["Pathogenicity_default"].values[0].string_value
print "Variant Id: {},\tVariant Set Id: {},\tReference Name: {}\n\tVariant Start: {},\tVariant End: {}\n\tReference Bases: {},\tAlternate Bases: {},\tClinical Significance: {}\n".format(SingleVar.id, SingleVar.variant_set_id,SingleVar.reference_name,SingleVar.start, SingleVar.end,SingleVar.reference_bases,SingleVar.alternate_bases,SigVar)
for i in SingleVar.info:
    print "{}: \t{}".format(i, SingleVar.info[str(i)].values[0].string_value or SingleVar.info[str(i)].values[0].number_value)

Variant Id: hg37-398,	Variant Set Id: brca-hg37,	Reference Name: chr17
	Variant Start: 41258469,	Variant End: 41258470
	Reference Bases: T,	Alternate Bases: [u'C'],	Clinical Significance: Pathogenic

Pathogenicity_research: 	Pathogenic(ENIGMA); Pathogenic (ClinVar); Pending (BIC)
Literature_source_exLOVD: 	Easton DF et al., Am J Hum Genet, 81: 873-883, 2007.
ClinVarAccession_ENIGMA: 	SCV000244314
Discordant: 	Concordant
Condition_category_ENIGMA: 	Disease
Date_Last_Updated_ClinVar: 	2015-08-10,2010-02-17
Reference_Name: 	chr17
Variant_in_LOVD: 	0.0
Segregation_LR_exLOVD: 	1
Source_URL: 	http://hci-exlovd.hci.utah.edu/variants.php?select_db=BRCA1&action=search_all&search_Variant%2FDNA=c.212+3A>G, http://www.ncbi.nlm.nih.gov/clinvar/?term=SCV000244314, http://www.ncbi.nlm.nih.gov/clinvar/?term=SCV000115252
Source: 	ENIGMA,ClinVar,LOVD,BIC,exLOVD
Condition_ID_value_ENIGMA: 	BREAST-OVARIAN CANCER, FAMILIAL, SUSCEPTIBILITY TO, 1; BROVCA1 (604370)
HGVS_Protein: 	NP_009225.1:p.?
Co_occurrence